In [1]:
import re
import xml.etree.ElementTree as ET

In [25]:
tree = ET.parse('LM_backend_challenge/C.xml')
root = tree.getroot()
print(root)

<Element '{http://www.abbyy.com/FineReader_xml/FineReader10-schema-v1.xml}document' at 0x00000157B3A84C70>


In [ ]:
def namespace(element):
    m = re.match(r'\{.*\}', element.tag)
    return m.group(0) if m else ''

In [3]:
def all_children(root):
    if 'formatting' in root.tag and root.text is not None:
        return root.text.strip()
    text = ''
    for child in root:
        temp = ''
        if not text:
            text = all_children(child)
        else:
            temp = all_children(child)
        if temp:
            text += '\n' + temp
    return text

In [4]:
def parse_text(root):
    block = []
    for item in root.findall('.//{http://www.abbyy.com/FineReader_xml/FineReader10-schema-v1.xml}text/'):
        sub_block = []
        for child in item.findall('.//{http://www.abbyy.com/FineReader_xml/FineReader10-schema-v1.xml}formatting'):
            if child.text:
                sub_block.append(child.text)
        if len(sub_block) > 0:
            block.append(sub_block)
    return block

In [3]:
def parse_text2(root):
    block = []
    for item in root.findall('.//{http://www.abbyy.com/FineReader_xml/FineReader10-schema-v1.xml}formatting'):
        if item.text:
            block.append(item.text)
    return block

In [109]:
text_list = []
for item in root.findall('.//{http://www.abbyy.com/FineReader_xml/FineReader10-schema-v1.xml}text/'):
    text_list.append(all_children(item))

In [4]:
def find_text(text_list, text):
    for index, value in enumerate(text_list):
        if text in value:
            return index

In [5]:
def find_text_reverse(text_list, text, start_index):
    for index in range(start_index, -1, -1):
        if text in text_list[index]:
            return index

In [6]:
def find_vs(text_list, defend_index):
    for index in range(defend_index, -1, -1):
        if re.search('[Vv][Ss]{0,1}\.', text_list[index]):
            return index

In [7]:
def find_relevant(text_list, start, end):
    text = ''
    for index in range(start, end-1, -1):
        if ',' in text_list[index] and len(text_list[index]) > 3:
            text += text_list[index] + '\n'
    return text

In [14]:
def find_defendant(text_list, start, end):
    text = ''
    for index in range(start, end + 1):
        if ',' in text_list[index] and len(text_list[index]) > 3:
            text += text_list[index] + '\n'
    return text

In [16]:
def find_plaintiff(text_list, start):
    for index in range(start - 1, -1, -1):
        if ',' in text_list[index] and len(text_list[index]) > 3:
            return text_list[index]

In [26]:
parse_list = parse_text2(root)
defend_index = find_text(parse_list, 'Defendant')
vs_index = find_vs(parse_list, defend_index)
plaint_index = find_text_reverse(parse_list, 'Plaintiff,', vs_index)
print(parse_list[plaint_index])
print(parse_list[vs_index])
print(parse_list[defend_index])

Plaintiff,
vs.
Defendants.


In [27]:
print(find_defendant(parse_list, vs_index, defend_index))
print(find_plaintiff(parse_list, plaint_index))

LAGUARDIA ENTERPRISES, INC., a
California Corporation, dba SONSONATE
GRILL; and DOES 1 through 25, inclusive,

ALBA ALVARADO, an individual;


In [31]:
parse_list = parse_text(root)
defend = False
plaint = False
defendant = 0
plaintiff = 0
plaint_index = find_text(parse_list, 'Plaintiff,')
defend_index = find_text(parse_list, 'Defendant')
for index, element in reversed(list(enumerate(parse_list))):
    for text in reversed(element):
        if defend and ',' in text:
            defendant = index
            defend = False
        if plaint and ',' in text:
            plaintiff = index
            plaint = False
        if defendant == 0 and 'Defendant' in text:
            defend = True
        if plaintiff == 0 and 'Plaintiff' in text:
            plaint = True
print('***')
for element in parse_list[defendant]:
    if 'Defendant' not in element:
        print(element)
print('***')
for element in parse_list[plaintiff]:
    if 'Plaintiff' not in element:
        print(element)

***
a California limited liability corporation, which owns and operates a property management
LLC owns and/or manages various residential properties in Alameda County.
***
a California limited liability corporation, which owns and operates a property management
LLC owns and/or manages various residential properties in Alameda County.


In [110]:
print(len(text_list))

90


In [111]:
for index, text in enumerate(text_list):
    print(index, text)

0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8
8 9
9 10
10 11
11 12
12 13
13 14
14 15
15 16
16 17
17 18
18 19
19 20
20 21
21 22
22 23
23 24
24 25
25 LAW OFFICES OF C. JOE SAYAS, JR.
26 C. JOE SAYAS, JR. (Bar No. 122397)
27 KARL P. EVANGELISTA (Bar No. 250685)
500 N. Brand Boulevard, Suite 980
28 Glendale, California 91203
Telephone: (818) 291-0088
Facsimile: (818) 240-9955
29 <ifU\
30 Attorneys for Plaintiff Angelo Angeles
31 FILED
32 SUPERIOR COURT OFCAUFDRNM
COUNTYOFLOSANGFlST
33 JAN
312017
34 Sherri R. Cartefcgecutive Officer/Clerk
-
DeTO
'
35 SUPERIOR COURT OF CALIFORNIA
36 COUNTY OF LOS ANGELES, CENTRAL DISTRICT
37 ANGELO ANGELES, an individual,
Plaintiff,
38 ) Case No.:
39 )
40 B C
6
4 8 7 4 4
41 ) COMPLAINT FOR:
42 )
43 vs.
44 )
)
45 HILL-ROM COMPANY, INC., an Indiana )
corporation; and DOES 1 through 100, inclusive, )
46 )
47 1.
48 2
.
49 Defendants.
50 b
51 3.
52 4.
53 DISABILITY DISCRIMINATION-
UNLAWFUL TERMINATION
(Cal. Gov.
Code § 12940(a
));
54 DISABILITY DISCRIMINATION—
RETALIATION FOR 

In [9]:
with open('LM_backend_challenge/A.xml', 'rb') as f:
    print(f)

<_io.BufferedReader name='LM_backend_challenge/A.xml'>
